# Recursive Clustering and Summarization

Plan:
- recursively cluster collections
- create tree of clusters (the HDBSCAN does this anyways but likely not as we want)
- cluster until max-depth is reached or (better) until each leaf only has one "plausible" cluster (based on thresholds or probabilities)
- try summarizing to get "main idea" out of cluster


- cluster on keywords ( randomize all grammar + stop words )
- topic clusters
- context: title, abstract, etc. keywords



## Recursively cluster 

Based on the topic_clustering notebook, we will try with Agglomerative Clustering 

In [45]:
import pandas as pd

df = pd.read_csv("downloads/40k_balanced_pm_acl.csv")#.sample(frac=0.5)

In [46]:
pos = df[df.labels == 1]
sentences = list(pos["text"]) #otherwise key error

In [47]:
len(sentences)

21835

In [48]:
from sentence_transformers import SentenceTransformer, util

print("Encode the corpus ... get a coffee in the meantime")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

Encode the corpus ... get a coffee in the meantime


Batches:   0%|          | 0/342 [00:00<?, ?it/s]

In [49]:
len(embeddings)

21835

In [50]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

def cluster(embeddings, **kwargs):
    embeddings = embeddings.cpu()
    # Normalize the embeddings to unit length
    corpus_embeddings = embeddings /  np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Perform kmean clustering
    clustering_model = AgglomerativeClustering(**kwargs) #, affinity='cosine', linkage='average', distance_threshold=0.4)
    clustering_model.fit(corpus_embeddings)
   # cluster_assignment = clustering_model.labels_
    return clustering_model

In [51]:
def get_clusters(clustering_model):
    
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering_model.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        try:
            clusters[cluster_id].append(sentences[sentence_id])
        except:
            print(sentence_id, "sentence_id")
    return clusters
   
#     for i, cluster in clustered_sentences.items():
#         print("Cluster ", i+1)
#         print(cluster)
#         print("\n")

In [52]:
sample = embeddings

In [53]:
cluster_model = cluster(sample, n_clusters=None, distance_threshold=1.4)

Cluster Model attributes

   n_clusters_ : int
        The number of clusters found by the algorithm. If
        ``distance_threshold=None``, it will be equal to the given
        ``n_clusters``.

    labels_
    n_leaves_

    n_connected_components_ : The estimated number of connected components in the graph.

    children_ : array-like of shape (n_samples-1, 2)
        The children of each non-leaf node. Values less than `n_samples`
        correspond to leaves of the tree which are the original samples.
        A node `i` greater than or equal to `n_samples` is a non-leaf
        node and has children `children_[i - n_samples]`. Alternatively
        at the i-th iteration, children[i][0] and children[i][1]
        are merged to form node `n_samples + i`

    distances_ : array-like of shape (n_nodes-1,)
        Distances between nodes in the corresponding place in `children_`.
        Only computed if `distance_threshold` is used or `compute_distances`
        is set to `True`.

In [13]:
# import itertools

# ii = itertools.count(sample.shape[0])
# [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in cluster_model.children_]

In [ ]:
len(get_clusters(cluster_model).keys())

In [ ]:
clusters = get_clusters(cluster_model)

In [ ]:
import matplotlib.pyplot as plt

for thresh in [1, 1.2, 1.4, 1.8]:
    cluster_model = cluster(sample, n_clusters=None, distance_threshold=thresh)
    plt.hist([len(v) for v in get_clusters(cluster_model).values()], bins='auto', label=str(thresh))
    plt.title("Threshold " + str(thresh))

In [ ]:
def collection_to_clusters(texts, model=model,  **kwargs):
    embs = model.encode(texts, batch_size=64, show_progress_bar=True, convert_to_tensor=True)
    cluster_model = cluster(embs, **kwargs)
    return get_clusters(cluster_model).values()

In [ ]:
cluster_tree = {}
lens = [len(v) for v in clusters.values()]
for i, v in clusters.items():
    if len(v) > 25:
        cluster_tree[i] = {"parent" : v}
        cluster_tree[i] = {"children" :[*collection_to_clusters(v, n_clusters=None, distance_threshold=0.4)]}
        #get values, embed and sample again with lower threshold
        

In [ ]:
for i,v in cluster_tree.items():
    print(i)
    for sent in v["children"]:
        print(sent)
        print("\n\n")
    print("-------------- \n\n")

In [66]:
#dict(enumerate(cluster_model.children_, cluster_model.n_leaves_))

## Summarization

**Tried: Google Pegasus**. Result: Does a terrible job of keeping the important information and doesn't retain the question but guesses at a conclusion


In [35]:
torch.cuda.is_available()

NameError: name 'torch' is not defined

### Pegasus Setup


In [36]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)


Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [37]:

def summarize(sentences):
    batch = tokenizer(sentences, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

### T5 Setup

In [39]:
from transformers import pipeline

summarizer = pipeline("summarization")

ARTICLE = """ Background: Trust is a critical component of competency committees given their high-stakes decisions. Research from outside of medicine on group trust has not focused on trust in group decisions, and "group trust" has not been clearly defined. The purpose was twofold: to examine the definition of trust in the context of group decisions and to explore what factors may influence trust from the perspective of those who rely on competency committees through a proposed group trust model. Methods: The authors conducted a literature search of four online databases, seeking articles published on trust in group settings. Reviewers extracted, coded, and analyzed key data including definitions of trust and factors pertaining to group trust. Results: The authors selected 42 articles for full text review. Although reviewers found multiple general definitions of trust, they were unable to find a clear definition of group trust and propose the following: a group-directed willingness to accept vulnerability to actions of the members based on the expectation that members will perform a particular action important to the group, encompassing social exchange, collective perceptions, and interpersonal trust. Additionally, the authors propose a model encompassing individual level factors (trustor and trustee), interpersonal interactions, group level factors (structure and processes), and environmental factors."""
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("snrspeaks/t5-one-line-summary") #snrspeaks/t5-one-line-summary
tokenizer = AutoTokenizer.from_pretrained("snrspeaks/t5-one-line-summary")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
tokenizer.decode(outputs[0])
def summarize(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

In [40]:
summarize("I have seen a ghost in my shed")

"<pad> Ghosts in sheds: a ghost in a ghost's kiloparsec-parsec-parsec-parsec-parsec-parsec-parsec-parsec</s>"

In [41]:
"I have seen a ghost in my shed"[:5]

'I hav'

### Print Results

In [42]:
for ID, cluster in get_clusters(cluster_model).items():
    sentences = ".".join(cluster)
    print(sentences)
    print( "\n\n", "sum:::", summarize(sentences[:256]), "\n\n\n")
    

The difficulty with this task lies in the fact that prosodic cues are never absolute ; they are relative to individual speakers , gender , dialect , discourse context , local context , phonological environment , and many other factors.Apart from system delay , another current limitation that will influence future interactive speech systems is the unavailability of full prosodic analysis.One shortcoming of LCSeg is that it ignores speaker information when segmenting transcripts.One obvious shortcoming is that some information gets lost in the thresholding that converts posterior probabilities from the prosodic model and the auxiliary LM into binary features.One major time and cost limitation in developing LVCSR systems in Indian languages is the need for large training data.While voice selection mode allows the presenter greater mobility , it has the drawback of allowing the audience to see thumbnails of every slide returned by a content-based query , regardless of whether the presenter